In [ ]:
import rasterio as rio
from tensorflow.keras.models import load_model
from core.optimizers import adaDelta
from core.losses import (
    tversky,
    accuracy,
    dice_coef,
    dice_loss,
    specificity,
    sensitivity,
)

In [ ]:
from importlib import reload
from config import RasterAnalysis as config
import scripts.raster_analysis as ras

ras = reload(ras)
config = reload(config)
config = config.Configuration()

In [ ]:
# Load a pretrained model
OPTIMIZER = adaDelta

# Get the last model
model_path = list(config.model_path.glob("*.h5"))[-1]
model = load_model(
    model_path,
    custom_objects={
        "tversky": tversky,
        "dice_coef": dice_coef,
        "dice_loss": dice_loss,
        "accuracy": accuracy,
        "specificity": specificity,
        "sensitivity": sensitivity,
    },
    compile=False,
)
model.compile(
    optimizer=OPTIMIZER,
    loss=tversky,
    metrics=[dice_coef, dice_loss, accuracy, specificity, sensitivity],
)

In [ ]:
# Predict trees in the all the files in the input image dir
# Depending upon the available RAM, images may not to be split before running this cell.
# Use the Auxiliary-2-SplitRasterToAnalyse if the images are too big to be analysed in memory.
all_files = list(config.input_image_dir.glob(f"*{config.input_image_type}"))
all_files;

In [ ]:
for image_path in all_files:
    
    output_file = config.output_dir/image_path.name
    
    with rio.open(image_path) as img:
        
        detectedMask, detectedMeta = ras.detect_tree(
            model,
            img,
            width=config.WIDTH,
            height=config.HEIGHT,
            stride=config.STRIDE,
        ) 
        
        break
        # WIDTH and HEIGHT should be the same and in this case Stride is 50 % width
        # Write the mask to file
        # ras.writeMaskToDisk(
        #     detectedMask,
        #     detectedMeta,
        #     outputFile,
        #     write_as_type=config.output_dtype,
        #     th=0.5,
        #     create_countors=False,
        # )

In [ ]:
# Display extracted image
sampleImage = ""
fn = os.path.join(config.output_dir, config.output_prefix + sampleImage)
predicted_img = rasterio.open(fn)
p = predicted_img.read()
np.unique(p, return_counts=True)
plt.imshow(p[0])